In [ ]:
import torch
import numpy as np
import os
os.chdir('..')

import torch_converter as tc
import instance_generator as ig
from torch_geometric.loader import DataLoader
from gnn_library.util import train, save, load
from util import NumpyDataset, collect
from evaluate import evaluate_model, pp_output

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("PyTorch has version {}".format(torch.__version__))
print('Using device:', device)

In [ ]:
GNN1, args1 = load('GNN1')
GNN2, args2 = load('GNN2')

In [ ]:
train_num = 50; test_num = 10

er_config = {
    'graph_type': 'ER',
    'p': 1,
    'weighted': True
}
ba_config = {
    'graph_type': 'BA',
    'ba_param': 4,
    'weighted': False
}
geom_config = {
    'graph_type': 'GEOM',
    'threshold': 0.2,
    'scaling': 1 / np.sqrt(2)
}

rng = np.random.default_rng()

train_instances = [
    *ig.sample_instances(6, 10, train_num, rng, **er_config),
    *ig.sample_instances(6, 10, train_num, rng, **ba_config),
    *ig.sample_instances(6, 10, train_num, rng, **geom_config),
    *ig.sample_instances(8, 8, train_num, rng, **er_config),
    *ig.sample_instances(8, 8, train_num, rng, **ba_config),
    *ig.sample_instances(8, 8, train_num, rng, **geom_config),
    *ig.sample_instances(10, 6, train_num, rng, **er_config),
    *ig.sample_instances(10, 6, train_num, rng, **ba_config),
    *ig.sample_instances(10, 6, train_num, rng, **geom_config)
]


test_instances = [
    *ig.sample_instances(6, 10, test_num, rng, **er_config),
    *ig.sample_instances(6, 10, test_num, rng, **ba_config),
    *ig.sample_instances(6, 10, test_num, rng, **geom_config),
    *ig.sample_instances(8, 8, test_num, rng, **er_config),
    *ig.sample_instances(8, 8, test_num, rng, **ba_config),
    *ig.sample_instances(8, 8, test_num, rng, **geom_config),
    *ig.sample_instances(10, 6, test_num, rng, **er_config),
    *ig.sample_instances(10, 6, test_num, rng, **ba_config),
    *ig.sample_instances(10, 6, test_num, rng, **geom_config)
]


train_dataset = NumpyDataset(
    tc._instances_to_nn_samples(
        instances=train_instances,
        models=[GNN1, GNN2],
        args=args1,
        batch_size=50,
        rng=rng
    )
)

test_dataset = NumpyDataset(
    tc._instances_to_nn_samples(
        instances=test_instances,
        models=[GNN1, GNN2],
        args=args1,
        batch_size=50,
        rng=rng
    )
)

args = {
    'processor':         'NN',
    'head':              'meta',
    'num_layers':        5,
    'input_dim':         17,
    'batch_size':        4,
    'hidden_dim':        16,
    'output_dim':        3,
    'dropout':           0,
    'epochs':            50,
    'opt':               'adam',
    'opt_scheduler':     'none',
    'opt_restart':       0,
    'weight_decay':      5e-3,
    'lr':                0.0001,
    'device':            device
}

train_loader = DataLoader(
    train_dataset,
    batch_size=args['batch_size'],
    shuffle=True,
    num_workers=4
)

test_loader = DataLoader(
    test_dataset,
    batch_size=args['batch_size'],
    shuffle=True,
    num_workers=4
)

In [ ]:
args = {
    'processor':         'NN',
    'head':              'meta',
    'num_layers':        5,
    'input_dim':         17,
    'batch_size':        4,
    'hidden_dim':        18,
    'output_dim':        3,
    'dropout':           0,
    'epochs':            50,
    'opt':               'adam',
    'opt_scheduler':     'none',
    'opt_restart':       0,
    'weight_decay':      5e-3,
    'lr':                0.0005,
    'device':            device
}

_, _, META_NN, _ = train(train_loader, test_loader, args)

In [ ]:
save(META_NN, args, 'META_NN')

In [ ]:
META_NN, args = load('META_NN')

In [ ]:
import evaluate as ev
seed = 0
(m, n) = (18, 6)
config = er_config

rng = np.random.default_rng(seed)
eval_instances = ig.sample_instances(m, n, 10, rng, **config)

ratios1 = evaluate_model(
    classify_model=META_NN,
    eval_models=[GNN1, GNN2],
    args=args,
    instances=eval_instances,
    batch_size=50,
    rng=rng,
    num_realizations=1
)

pp_output(ratios1, _, show_log=False)
print()
print()

rng = np.random.default_rng(seed)
eval_instances = ig.sample_instances(m, n, 10, rng, **config)

ratios2 = evaluate_model(
    classify_model=None,
    eval_models=[GNN1],
    args=args,
    instances=eval_instances,
    batch_size=50,
    rng=rng,
    num_realizations=10
)

pp_output(ratios2, _, show_log=False)

print()
print()

rng = np.random.default_rng(seed)
eval_instances = ig.sample_instances(m, n, 10, rng, **config)

ratios = evaluate_model(
    classify_model=None,
    eval_models=[GNN2],
    args=args,
    instances=eval_instances,
    batch_size=50,
    rng=rng,
    num_realizations=10
)

pp_output(ratios, _, show_log=False)

In [ ]:
(m, n) = (11, 16)
rng = np.random.default_rng(10)
eval_instances = ig.sample_instances(m, n, 10, rng, **er_config)

X = tc._instances_to_nn_eval(eval_instances).to('cuda')
print(META_NN((X, None)))


In [ ]:
rng = np.random.default_rng(10)
eval_instances, eval_coin_flips = ig.sample_instances(32, 32, 10, rng, **ba_config)

ratios1, log = evaluate_model(
    model=GNN1,
    args=args,
    instances=eval_instances,
    coin_flips=eval_coin_flips,
    batch_size=50,
    rng=rng
)

In [ ]:
rng = np.random.default_rng(10)
eval_instances, eval_coin_flips = ig.sample_instances(32, 32, 10, rng, **ba_config)

ratios2, log = evaluate_model(
    model=GNN2,
    args=args,
    instances=eval_instances,
    coin_flips=eval_coin_flips,
    batch_size=50,
    rng=rng
)

In [ ]:
num_trials = 100
node_configs = [(x, 16) for x in np.arange(4, 64, 2)]
# of nodes [20 -> 80]
# of nodes in batch [10,000 -> 40,000]
batch_size = 100 #[int(min(32, x + y)) for (x, y) in node_configs]
graph_configs = [
    # {
    #     'graph_type': 'GM'
    # },
    {
        'graph_type': 'ER',
        'p': 0.75,
        'weighted': True
    },
    {
        'graph_type': 'BA',
        'ba_param': 4,
        'weighted': True
    },
    {
        'graph_type': 'GEOM',
        'threshold': 0.2,
        'scaling': 1 / np.sqrt(2),
    }
]

ratios = [x/y for (x,y) in node_configs]
print(ratios)
data = {config['graph_type']: [] for config in graph_configs}
for graph_config in graph_configs:
    for i, node_config in enumerate(node_configs):
        seed = np.random.randint(0, 500000)
        rng = np.random.default_rng(seed)
        instances = ig.sample_instances(*node_config, num_trials, rng, **graph_config)

        rng = np.random.default_rng(seed)
        (meta_learned_ratios, _), _ = evaluate_meta_model(
            classify_model=META_NN,
            eval_models=[GNN1, GNN2],
            args=args,
            instances=instances,
            batch_size=50,
            rng=rng,
            
        )

        rng = np.random.default_rng(seed)
        (gnn1_learned_ratios, greedy_ratios), _ = evaluate_model(
            GNN1,
            args,
            instances,
            batch_size,
            rng,
            num_realizations=1
            )

        rng = np.random.default_rng(seed)
        (gnn2_learned_ratios, _), _ = evaluate_model(
            GNN2,
            args,
            instances,
            batch_size,
            rng,
            num_realizations=1
        )

        # print(np.array(
        #     [
        #         meta_learned_ratios,
        #         gnn1_learned_ratios,
        #         gnn2_learned_ratios,
        #         greedy_ratios
        #     ]
        # ))
        data[graph_config['graph_type']].append(np.array(
            [
                meta_learned_ratios,
                gnn1_learned_ratios,
                gnn2_learned_ratios,
                greedy_ratios
            ]
        ))

In [ ]:
import matplotlib.pyplot as plt
for graph_type, comp_ratios in data.items():
    greedy_avg_ratios = []
    meta_avg_ratios = []
    gnn1_avg_ratios = []
    gnn2_avg_ratios = []
    max_avg_ratios = []

    for trial_ratios in comp_ratios:
        meta_avg_ratios.append(np.array(trial_ratios[0]).mean())
        gnn1_avg_ratios.append(np.array(trial_ratios[1]).mean())
        gnn2_avg_ratios.append(np.array(trial_ratios[2]).mean())
        greedy_avg_ratios.append(np.array(trial_ratios[3]).mean())
        max_avg_ratios.append(np.array(np.max(trial_ratios[:, 1:], axis=0)).mean())

    print(graph_type)
    fig = plt.figure(figsize=(8,6))
    plt.title(graph_type)
    plt.plot(ratios, gnn1_avg_ratios, label='GNN1')
    plt.plot(ratios, gnn2_avg_ratios, label='GNN2')
    plt.plot(ratios, greedy_avg_ratios, label='Greedy')
    plt.plot(ratios, max_avg_ratios, label='MAX')
    plt.plot(ratios, meta_avg_ratios, label='META')
    plt.xlabel('# online / # offline')
    plt.ylabel('Average competitive ratio')
    plt.legend()
    plt.show()


In [ ]:
seed = np.random.randint(0, 500000)
rng = np.random.default_rng(seed)

ba_config = {
    'graph_type': 'BA',
    'ba_param': 2,
    'weighted': False
}

instances = ig.sample_instances(*(12, 8), 100, rng, **ba_config)
batch_size = 50

rng = np.random.default_rng(seed)
(gnn1_learned_ratios, greedy_ratios), _ = evaluate_model(
    GNN1,
    args,
    instances,
    batch_size,
    rng,
    num_realizations=1
    )

rng = np.random.default_rng(seed)
(gnn2_learned_ratios, _), _ = evaluate_model(
    GNN2,
    args,
    instances,
    batch_size,
    rng,
    num_realizations=1
)

In [ ]:
comp = np.vstack([gnn1_learned_ratios, gnn2_learned_ratios, greedy_ratios]).T
comp[:10]

In [ ]:
print(np.max(comp, axis=1).mean())
print(comp.mean(axis=0))

In [ ]:
index2 = np.argmax(comp[:, 1] - comp[:, 0])

In [ ]:
index1 = np.argmax(comp[:, 0] - comp[:, 1])

In [ ]:
tc._featurize(instances[index1])

In [ ]:
tc._featurize(instances[index2])

In [ ]:
GNN2_features = []
GNN1_features = []
for i, boolean in enumerate(comp[:, 1] > comp[:, 0]):
    if boolean:
        GNN2_features.append(tc._featurize(instances[i]))
    else:
        GNN1_features.append(tc._featurize(instances[i]))

out1 = np.vstack(GNN1_features)
out2 = np.vstack(GNN2_features)


In [ ]:
out1.mean(axis=0)

In [ ]:
out2.mean(axis=0)